In [1]:
pip install -r /config/workspace/requirements.txt

     |████████████████████████████████| 9.2 MB 21.8 MB/s eta 0:00:01
     |████████████████████████████████| 479.6 MB 23 kB/s  eta 0:00:012     |█████████████████████████████▎  | 438.7 MB 519 kB/s eta 0:01:19
  Using cached seaborn-0.13.0-py3-none-any.whl (294 kB)
     |████████████████████████████████| 11.1 MB 499 kB/s eta 0:00:011
     |████████████████████████████████| 4.7 MB 481 kB/s eta 0:00:011
     |████████████████████████████████| 301 kB 31.0 MB/s eta 0:00:01
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
     |████████████████████████████████| 17.3 MB 495 kB/s eta 0:00:01
  Using cached importlib_resources-6.1.1-py3-none-any.whl (33 kB)
     |████████████████████████████████| 3.5 MB 14.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 66.7 MB/s eta 0:00:01
  Using cached pyparsing-3.1.1-py3-none-any.whl (103 kB)
     |████████████████████████████████| 83 kB 2.0 MB/s eta 0:00:011
  Using cached absl_py-2.0.0-py3-none-any.whl (130 kB)
     |█████████████

In [2]:
import numpy as np
import tensorflow as tf 
from tensorflow import keras
import pandas as pd
from matplotlib import  pyplot as plt
%matplotlib inline

2023-12-07 14:31:35.934492: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-07 14:31:36.007176: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-07 14:31:36.009000: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-07 14:31:37.241974: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
df=pd.read_csv("insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [4]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2,random_state=42)

In [5]:
## Preprocessing of the data
x_train_scaled=x_train.copy()
x_train_scaled['age']=x_train_scaled.age/100

x_test_scaled=x_test.copy()
x_test_scaled.age=x_test_scaled.age/100

##### Model Building: First build a model in keras/tensorflow and see what weights and bias values it comes up with. We will than try to reproduce same weights and bias in our plain python implementation of gradient descent. Below is the architecture of our simple neural network

In [6]:
model=keras.Sequential([
    keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',kernel_initializer='ones',bias_initializer='zeros')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.fit(x_train_scaled,y_train,epochs=1000)

Epoch 1/1000
1/1 [==============================] - 0s 489ms/step - loss: 0.7428 - accuracy: 0.5000
Epoch 2/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7424 - accuracy: 0.5000
Epoch 3/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7420 - accuracy: 0.5000
Epoch 4/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7416 - accuracy: 0.5000
Epoch 5/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7411 - accuracy: 0.5000
Epoch 6/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7407 - accuracy: 0.5000
Epoch 7/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7403 - accuracy: 0.5000
Epoch 8/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7399 - accuracy: 0.5000
Epoch 9/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7395 - accuracy: 0.5000
Epoch 10/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7390 - accuracy: 0.5000
Epoch 11/1000
1/1

In [7]:
## evaluate the model on the test cases 
model.evaluate(x_test_scaled,y_test)

1/1 [==============================] - 0s 164ms/step - loss: 0.5276 - accuracy: 0.8333


[0.5275540351867676, 0.8333333134651184]

In [8]:
model.predict(x_test_scaled)

1/1 [==============================] - 0s 80ms/step


array([[0.65436   ],
       [0.6362884 ],
       [0.65690714],
       [0.41346118],
       [0.54941016],
       [0.41619965]], dtype=float32)

In [9]:
y_test

9     1
25    1
8     1
21    0
0     0
12    0
Name: bought_insurance, dtype: int64

In [10]:
## Extracting the values of weights and bias from the model
coef,intercept=model.get_weights()

In [11]:
coef,intercept

(array([[1.1281339],
        [0.5930885]], dtype=float32),
 array([-0.6429901], dtype=float32))

In [12]:
## This means the array one is the array of weights and second array is the array of bias

In [13]:
def sigmoid(x):
    import math
    return 1/(1+math.exp(-x))
sigmoid(18)

0.9999999847700205

In [14]:
x_test

,age,affordibility
9,61,1
25,54,1
8,62,1
21,26,0
0,22,1
12,27,0


In [15]:
## insted o f model.predict we can write our owm prediction function that uses w1,w2 and bias
def prediction_function(age, affordibility):
    weighted_sum = coef[0]*age + coef[1]*affordibility + intercept
    return sigmoid(weighted_sum)

prediction_function(.47, 1)

0.6178237675542665

In [16]:
prediction_function(.18, 1)

0.538215940875218

In [17]:
## Now we start implementing gradient descent in plain python. Again the goal is to come up with same w1, w2 and bias that keras model calculated. We want to show how keras/tensorflow would have computed these values internally using gradient descent
# First write couple of helper routines such as sigmoid and log_loss
def sigmoid_numpy(X):
   return 1/(1+np.exp(-X))
   
sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [18]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [19]:
## All right now comes the time to implement our own custom neural network class !! yay !!!

class myNN:
    def __init__(self):
        self.w1 = 1 
        self.w2 = 1
        self.bias = 0
        
    def fit(self, X, y, epochs, loss_thresold):
        self.w1, self.w2, self.bias = self.gradient_descent(X['age'],X['affordibility'],y, epochs, loss_thresold)
        print(f"Final weights and bias: w1: {self.w1}, w2: {self.w2}, bias: {self.bias}")
        
    def predict(self, X_test):
        weighted_sum = self.w1*X_test['age'] + self.w2*X_test['affordibility'] + self.bias
        return sigmoid_numpy(weighted_sum)

    def gradient_descent(self, age,affordability, y_true, epochs, loss_thresold):
        w1 = w2 = 1
        bias = 0
        rate = 0.5
        n = len(age)
        for i in range(epochs):
            weighted_sum = w1 * age + w2 * affordability + bias
            y_predicted = sigmoid_numpy(weighted_sum)
            loss = log_loss(y_true, y_predicted)
            
            w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
            w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 

            bias_d = np.mean(y_predicted-y_true)
            w1 = w1 - rate * w1d
            w2 = w2 - rate * w2d
            bias = bias - rate * bias_d
            
            if i%50==0:
                print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
            
            if loss<=loss_thresold:
                print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
                break

        return w1, w2, bias

In [20]:
customModel = myNN()
customModel.fit(x_train_scaled, y_train, epochs=8000, loss_thresold=0.4631)

Epoch:0, w1:0.9736899318847281, w2:0.931388810977659, bias:-0.11748951666770448, loss:0.7428288579142563
Epoch:50, w1:1.524279872239132, w2:0.8822187836689879, bias:-1.1310088596841466, loss:0.5943161377198863
Epoch:100, w1:2.2281870363373972, w2:1.021207570480777, bias:-1.5219348336892933, loss:0.5674253359204207
Epoch:150, w1:2.8788802020633866, w2:1.0918966282424585, bias:-1.8376731064036336, loss:0.5462065237139112
Epoch:200, w1:3.473286412049912, w2:1.1354958313845407, bias:-2.109494376761035, loss:0.5289749850551891
Epoch:250, w1:4.014235251285109, w2:1.168307211546194, bias:-2.3515205001257873, loss:0.5148317335261617
Epoch:300, w1:4.506491403088423, w2:1.1967920767703706, bias:-2.570834600613761, loss:0.5031414005690805
Epoch:350, w1:4.955154772161112, w2:1.2234470550440064, bias:-2.7714756232427904, loss:0.4934177492028928
Epoch:400, w1:5.365041991551451, w2:1.2491715720776855, bias:-2.956078966702517, loss:0.48528128420193234
Epoch:450, w1:5.740481994774169, w2:1.274234883839

In [21]:
coef, intercept
## This shows that in the end we were able to come up with same value of w1,w2 and bias using a plain python implementation of gradient descent function

(array([[1.1281339],
        [0.5930885]], dtype=float32),
 array([-0.6429901], dtype=float32))

In [22]:
x_test_scaled

,age,affordibility
9,0.61,1
25,0.54,1
8,0.62,1
21,0.26,0
0,0.22,1
12,0.27,0


In [23]:
## predict using costum model
customModel.predict(x_test_scaled)

9     0.866457
25    0.802014
8     0.874054
21    0.137819
0     0.319842
12    0.146012
dtype: float64

In [24]:
## predict usign tensorflow model
model.predict(x_test_scaled)

1/1 [==============================] - 0s 26ms/step


array([[0.65436   ],
       [0.6362884 ],
       [0.65690714],
       [0.41346118],
       [0.54941016],
       [0.41619965]], dtype=float32)

In [25]:
##Above you can compare predictions from our own custom model and tensoflow model. You will notice that predictions are almost same